In [1]:
import pandas as pd
import yfinance as yf
from concurrent.futures import ThreadPoolExecutor

# Get Data

In [40]:
"""
Patents: 2,005,962 entries
wku = patent number
permco = company identifier
permno = stock (share class) identifier

Only counts patents from after 2000
"""

df_kpss = pd.read_stata("data/KPSS.dta")

df_kpss.rename(columns={'wku': 'patnum'}, inplace=True)
df_kpss['permco'] = pd.to_numeric(df_kpss['permco'], errors='coerce').astype('Int64')
df_kpss['permno'] = pd.to_numeric(df_kpss['permno'], errors='coerce').astype('Int64')
df_kpss = df_kpss.drop(['date','lmkcap', '_1982adjval', 'firmvol', 'Lfirmvol', 'firmvol15'], axis=1)
df_kpss.rename(columns={'year': 'YEAR','patnum': 'PATNUM', 'permno': 'PERMNO', 'permco':'PERMCO'}, inplace=True)
df_kpss = df_kpss.dropna()
df_kpss = df_kpss[df_kpss['YEAR'] >= 2000]

In [11]:
"""
Match permno to stock tickers 
~ 9000 tickers
"""

df_nasdaq = pd.read_csv("data/nasdaq_permno.csv")
df_nyse = pd.read_csv("data/nyse_permno.csv")
df_amex = pd.read_csv("data/amex_permno.csv")
df_ticker_permno = pd.concat([df_nasdaq, df_nyse, df_amex], axis=0)
df_ticker_permno = df_ticker_permno.drop(['DATE'], axis=1)
len(df_ticker_permno)

In [21]:
"""
info_type could be sector, marketcap, etc.
"""
cache = {}

def get_stock_info(ticker, info_type):
    if ticker in cache:
        return cache[ticker]
    try:
        yf_ticker = yf.Ticker(ticker)
        cache[ticker] = yf_ticker.info[info_type]
        return cache[ticker]
    except Exception as e:
        cache[ticker] = None
        return None

In [42]:
# If import data from files
df_patent_sector = pd.read_csv("data/patent_sector.csv")

df_market_cap = pd.read_csv("data/market_cap.csv")
df_market_cap = df_market_cap.drop(['Unnamed: 0'], axis=1)

In [12]:
"""
Create dataset mapping patents to tickers then sector
"""
df_patent_sector = pd.merge(df_kpss, df_ticker_permno, on='PERMNO', how='inner')
cache = {}
df_patent_sector["SECTOR"] = df_patent_sector['TICKER'].apply(get_stock_info, args=('sector',))
#df_ticker_patent.to_csv("data/patent_sector.csv")

In [ ]:
"""
Create dataset with tickers and market cap 
"""
df_market_cap = pd.DataFrame()
df_market_cap['TICKER'] = df_ticker_permno['TICKER']

cache = {}
df_market_cap['MARKET CAP'] = df_market_cap['TICKER'].apply(get_stock_info, args=('marketCap',))
#df_market_cap.to_csv("market_cap.csv")

# Create Factor

In [92]:
""" 
Create table with num patents and market cap for each ticker
"""

df = df_patent_sector.groupby(['TICKER','COMNAM','SECTOR']).size().reset_index(name='NUM PATENTS')
df = pd.merge(df_market_cap, df, on='TICKER', how='inner')
df['PATENT MCAP RATIO'] = df['NUM PATENTS']/df['MARKET CAP']

# Remove duplicates of company that might be due to different name
df = df[~df['TICKER'].duplicated(keep='first')]
df = df.set_index('TICKER')

In [67]:
# Find average patent/mcap ratio per sector
sector_avg = df.groupby('SECTOR')['PATENT MCAP RATIO'].mean()

In [94]:
# Create patent intensity factor
df['PATENT FACTOR'] = df.apply(lambda row: row['PATENT MCAP RATIO'] / sector_avg[row['SECTOR']], axis=1)
#df.to_csv("data/patent_factor.csv")

In [88]:
df.head(15) 

,MARKET CAP,COMNAM,SECTOR,NUM PATENTS,PATENT MCAP RATIO,PATENT FACTOR
TICKER,,,,,,
AAL,9.469809e+09,AMERICAN AIRLINES GROUP INC,Industrials,16,1.689580e-09,0.014965
AAON,5.777325e+09,AAON INC,Industrials,8,1.384724e-09,0.012265
AAPL,3.072767e+12,APPLE COMPUTER INC,Technology,1766,5.747263e-10,0.000418
ABEO,1.206460e+08,ABEONA THERAPEUTICS INC,Healthcare,4,3.315486e-08,0.079543
ABIO,2.521768e+07,ARCA BIOPHARMA INC,Healthcare,73,2.894795e-06,6.945007
ABL,5.099659e+08,AMERICAN BILTRITE INC,Financial Services,1,1.960915e-09,0.005926
ACAD,4.692264e+09,ACADIA PHARMACEUTICALS,Healthcare,40,8.524669e-09,0.020452
ACET,6.863156e+07,ACETO CORP,Healthcare,1,1.457056e-08,0.034957
ACHV,6.730725e+07,ACHIEVE LIFE SCIENCES INC,Healthcare,20,2.971448e-07,0.712891


In [90]:
aapl_factor = df.loc['AAPL', 'PATENT FACTOR']
aapl_factor

0.00041834004765317146